In this project you are going to predict the overall rating of soccer player based on their
attributes such as 'crossing', 'finishing etc.

In [1]:
import sqlite3
import pandas as pd 
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt


In [2]:
### reading data from database 
connect = sqlite3.connect('database.sqlite')
df = pd.read_sql_query('SELECT * FROM Player_Attributes',connect)

In [3]:
## drop useless columns 
df.drop(columns=['id','player_fifa_api_id','player_api_id','date'],inplace=True)

In [4]:
df.isnull().sum()

overall_rating          836
potential               836
preferred_foot          836
attacking_work_rate    3230
defensive_work_rate     836
crossing                836
finishing               836
heading_accuracy        836
short_passing           836
volleys                2713
dribbling               836
curve                  2713
free_kick_accuracy      836
long_passing            836
ball_control            836
acceleration            836
sprint_speed            836
agility                2713
reactions               836
balance                2713
shot_power              836
jumping                2713
stamina                 836
strength                836
long_shots              836
aggression              836
interceptions           836
positioning             836
vision                 2713
penalties               836
marking                 836
standing_tackle         836
sliding_tackle         2713
gk_diving               836
gk_handling             836
gk_kicking          

In [5]:
## drop/ fill missing value 
df.dropna(how='all',axis=0,inplace=True)

In [6]:
col_hav_missing = df.isnull().sum()[df.isnull().sum()>0].index

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183142 entries, 0 to 183977
Data columns (total 38 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   overall_rating       183142 non-null  float64
 1   potential            183142 non-null  float64
 2   preferred_foot       183142 non-null  object 
 3   attacking_work_rate  180748 non-null  object 
 4   defensive_work_rate  183142 non-null  object 
 5   crossing             183142 non-null  float64
 6   finishing            183142 non-null  float64
 7   heading_accuracy     183142 non-null  float64
 8   short_passing        183142 non-null  float64
 9   volleys              181265 non-null  float64
 10  dribbling            183142 non-null  float64
 11  curve                181265 non-null  float64
 12  free_kick_accuracy   183142 non-null  float64
 13  long_passing         183142 non-null  float64
 14  ball_control         183142 non-null  float64
 15  acceleration     

In [8]:
categorical_variables = df.dtypes [df.dtypes == 'O'].index

In [9]:
for col in categorical_variables:
    print(df[col].value_counts())

right    138409
left      44733
Name: preferred_foot, dtype: int64
medium    125070
high       42823
low         8569
None        3639
norm         348
y            106
le           104
stoc          89
Name: attacking_work_rate, dtype: int64
medium    130846
high       27041
low        18432
_0          2394
o           1550
1            441
ormal        348
2            342
3            258
5            234
7            217
0            197
6            197
9            152
4            116
es           106
ean          104
tocky         89
8             78
Name: defensive_work_rate, dtype: int64


In [10]:
create_missing_col = False
for col in col_hav_missing:
    if df[col].dtype == 'O':
        df[col].fillna('None',inplace = True)
    else:
        if not create_missing_col: 
            df['missing'] = np.where(df[col].isnull(),1,0) 
            create_missing_col = True
        df[col].fillna(df[col].median(),inplace = True)

In [11]:
col_hav_missing

Index(['attacking_work_rate', 'volleys', 'curve', 'agility', 'balance',
       'jumping', 'vision', 'sliding_tackle'],
      dtype='object')

In [12]:
(df['missing'] == 1).sum()

1877

In [13]:
## dealing with categorical variable 
categorical_variables = df.dtypes [df.dtypes == 'O'].index

df = pd.concat([df.drop(columns=categorical_variables),
                pd.get_dummies(df[categorical_variables],prefix=categorical_variables,drop_first=True)],axis = 1)

In [14]:
## split X and y
X = df.drop(columns=['overall_rating'])
y = df['overall_rating']

In [15]:
## correlation test using spearman correlation 
from scipy.stats import spearmanr
for col in X.columns:
    rho, pval = spearmanr(X[col], y)
    if pval>.05:
        print(col)
        del X[col]

gk_kicking
preferred_foot_right


In [16]:
## train test split 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = .3,random_state=2123)

In [17]:
## split for hyperparameter tunning 
X_train1,X_test1,y_train1,y_test1 = train_test_split(X,y,train_size = .3,random_state=1234)

In [18]:
## model building -- regression 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
params = {'penalty':'elasticnet','l1_ratio': 1, 'alpha': 0.01}
pipeLinear = Pipeline([('standard_scaling', StandardScaler()),('ElasticNet',SGDRegressor(**params))])
pipeLinear.fit(X_train,y_train)

Pipeline(steps=[('standard_scaling', StandardScaler()),
                ('ElasticNet',
                 SGDRegressor(alpha=0.01, l1_ratio=1, penalty='elasticnet'))])

In [19]:
pipeLinear.score(X_test,y_test)

0.8361921391801014

In [21]:
## model building -- decision tree 
regressor = DecisionTreeRegressor(random_state=0)
regressor.fit(X_train,y_train)

DecisionTreeRegressor(random_state=0)

In [22]:
regressor.score(X_test,y_test)

0.9593466565883333